In [1]:
# Configurar el path para importar los módulos
import sys
import os

# Añadir el directorio raíz del proyecto al path
root_dir = os.chdir(os.path.abspath(os.path.join(os.getcwd(), '..')))
if root_dir not in sys.path:
    sys.path.append(root_dir)


In [2]:
# Importar las funciones necesarias
from db.conexion_db import crear_conexion, cerrar_conexion
from db.querys_db import insert_data, consult_data
import pandas as pd
import numpy as np


In [3]:
connection = crear_conexion()
reservas = consult_data(connection,'select * from plan_reservas')

Conexion Exitosa


In [4]:
connection = crear_conexion()
ordenes = consult_data(connection,'select orden,claot,equipo,material,felib,orden_texto,orden_estsis,reservasolped,orden_prio,orden_prio_texto,orden_revision,estatususuarioorden from plan_ordenes')

Conexion Exitosa


In [5]:
connection = crear_conexion()
aviso_orden = consult_data(connection,'select * from plan_avisorden')

Conexion Exitosa


In [6]:
connection = crear_conexion()
material = consult_data(connection,'select material,Cod_mate_antiguo,Grp_mat,Tp_mat,Jerarquia_produc from tbl_materialesIH09')

Conexion Exitosa


In [7]:
tabla_reservas = pd.DataFrame(reservas)
tabla_ordenes = pd.DataFrame(ordenes)
tabla_aviso = pd.DataFrame(aviso_orden)
tabla_material = pd.DataFrame(material)

In [8]:
tabla_aviso.drop_duplicates(['orden'],inplace=True)
tabla_ordenes.drop_duplicates(['orden'],inplace=True)

In [9]:
tabla_ordenes.rename(columns={"material": "modelo", "orden_prio": "prio_orden","orden_prio_texto" :"prio_orden_texto"},inplace=True)
tabla_aviso.drop(columns=['id','fechacarga'],inplace=True)
tabla_reservas.drop(columns=['id','fechacarga'],inplace=True)

In [10]:
#Se eliminar las reservas que se encuentran borradas

filtrar_tbl_reservas = tabla_reservas.loc[(tabla_reservas['salida_fin']!='X') & (tabla_reservas['reserva_pos_borrado']!='X') & (tabla_reservas['relevplnec']!='X') & (tabla_reservas['orden']!='')].reset_index(drop=True)

In [11]:
unir_reservas_ordenes = filtrar_tbl_reservas.merge(tabla_ordenes,how='left',on='orden')


In [12]:
unir_reservaxorden_aviso = unir_reservas_ordenes.merge(tabla_aviso,how='left',on='orden')
unir_reservaxorden_aviso.where(pd.notna(unir_reservaxorden_aviso),None,inplace=True)

In [13]:
unir_reservaxorden_aviso['felib'] = np.where( unir_reservaxorden_aviso['felib_x'].isnull(), unir_reservaxorden_aviso['felib_y']  , unir_reservaxorden_aviso['felib_x'])
unir_reservaxorden_aviso['equipo'] = np.where( unir_reservaxorden_aviso['equipo_x'].isnull(), unir_reservaxorden_aviso['equipo_y']  , unir_reservaxorden_aviso['equipo_x'])
unir_reservaxorden_aviso['orden_texto'] = np.where( unir_reservaxorden_aviso['orden_texto_x'].isnull(), unir_reservaxorden_aviso['orden_texto_y']  , unir_reservaxorden_aviso['orden_texto_x'])
unir_reservaxorden_aviso['orden_revision'] = np.where( unir_reservaxorden_aviso['orden_revision_x'].isnull(), unir_reservaxorden_aviso['orden_revision_y']  , unir_reservaxorden_aviso['orden_revision_x'])
unir_reservaxorden_aviso['estatususuarioorden'] = unir_reservaxorden_aviso['estatususuarioorden_x']
unir_reservaxorden_aviso['detalle01'] = unir_reservaxorden_aviso['estatususuarioorden_y']
unir_reservaxorden_aviso['detalle02'] = unir_reservaxorden_aviso['claot']

unir_reservaxorden_aviso.drop(columns='felib_x', inplace=True)
unir_reservaxorden_aviso.drop(columns='felib_y', inplace=True)
unir_reservaxorden_aviso.drop(columns='equipo_x', inplace=True)
unir_reservaxorden_aviso.drop(columns='equipo_y', inplace=True)
unir_reservaxorden_aviso.drop(columns='orden_texto_x', inplace=True)
unir_reservaxorden_aviso.drop(columns='orden_texto_y', inplace=True)
unir_reservaxorden_aviso.drop(columns='orden_revision_x', inplace=True)
unir_reservaxorden_aviso.drop(columns='orden_revision_y', inplace=True)

In [14]:
unir_reservaxorden_aviso['ate_accion'] = ""
unir_reservaxorden_aviso['ate_orden'] = 999
unir_reservaxorden_aviso['valor_flota'] = 0.00
unir_reservaxorden_aviso['stock_asignado'] = 0.00
unir_reservaxorden_aviso['trasl_ctd_c154'] = 0.00
unir_reservaxorden_aviso['trasl_ctd_c040'] = 0.00
unir_reservaxorden_aviso['trasl_ctd_c080'] = 0.00
unir_reservaxorden_aviso['trasl_ctd_c200'] = 0.00
unir_reservaxorden_aviso['trasl_ctd_c152'] = 0.00
unir_reservaxorden_aviso['trasl_fecha'] = None
unir_reservaxorden_aviso['Ped_nro'] = ''
unir_reservaxorden_aviso['Ped_Fecha'] = None
unir_reservaxorden_aviso['Ped_ctd'] = 0.00
unir_reservaxorden_aviso['Ped_reserva'] = ''
unir_reservaxorden_aviso['Ped_nro_necesidad'] = ''
unir_reservaxorden_aviso['Ped_proveedor'] = ''
unir_reservaxorden_aviso['estadoreserva'] = ''

In [15]:
unir_reservaxorden_aviso[['ctd_dif','eqvaloradq_usd']] = unir_reservaxorden_aviso[['ctd_dif','eqvaloradq_usd']].astype(float)


In [16]:
unir_reservaxorden_aviso.loc[unir_reservaxorden_aviso['eqvaloradq_usd'].isna(),'eqvaloradq_usd'] = 1

In [17]:
unir_reservaxorden_aviso['estadoreserva'] = np.where( (unir_reservaxorden_aviso['orden'] != "") & (unir_reservaxorden_aviso['ot_liberado'] == 'X') & (unir_reservaxorden_aviso['reserva_pos_borrado'] != 'X') ,"Lib Mov.Permitdo", unir_reservaxorden_aviso['estadoreserva'])
unir_reservaxorden_aviso['estadoreserva'] = np.where( (unir_reservaxorden_aviso['orden'] != "") & (unir_reservaxorden_aviso['ot_liberado'] == '') & (unir_reservaxorden_aviso['reservasolped'] == '3') & (unir_reservaxorden_aviso['reserva_pos_borrado'] != 'X') ,"Aprob Mov.Permitdo", unir_reservaxorden_aviso['estadoreserva'])
unir_reservaxorden_aviso['estadoreserva'] = np.where( (unir_reservaxorden_aviso['orden'] == "") & (unir_reservaxorden_aviso['reserva_pos_borrado'] != 'X') ,"Lib Mov.Permitdo", unir_reservaxorden_aviso['estadoreserva'])

In [18]:
# DETERMINAR ACCION de Abastecimiento
# 1. Revisamos los Servicios
unir_reservaxorden_aviso.loc[unir_reservaxorden_aviso['um_base'] == "ZZ", 'ate_accion'] = "SERVICIO"
# 2. Revisamos las Reparaciones
unir_reservaxorden_aviso.loc[unir_reservaxorden_aviso['lote'] == "REPARADO", 'ate_accion'] = "REPARADO"
# 3. Revisamos la Mercaderia
unir_reservaxorden_aviso.loc[unir_reservaxorden_aviso['ate_accion'] == "", 'ate_accion'] = "MERCADERIA"
# 4. Anulamos los que no se deben atender: GARANTIA ubicado en el campo LOTE/Reservas Borradas/Reservas Atendidas o con Orden Cerrada campo salida_fin
unir_reservaxorden_aviso.loc[unir_reservaxorden_aviso['salida_fin'] == "X" , 'ate_accion'] = "FIN"
unir_reservaxorden_aviso.loc[unir_reservaxorden_aviso['reserva_pos_borrado'] == "X", 'ate_accion'] = "FIN"
unir_reservaxorden_aviso.loc[unir_reservaxorden_aviso['lote'] == "GARANTIA" , 'ate_accion'] = "FIN"
unir_reservaxorden_aviso.loc[unir_reservaxorden_aviso['reservasolped'] == "2" , 'ate_accion'] = "FIN"

In [19]:
# Personas -> EPPs y Uniformes
unir_reservaxorden_aviso.loc[unir_reservaxorden_aviso['orden'].str[0] =='P','ate_orden'] = 100

In [20]:
# STC

# Equipos -> Ordenes de trabajo Libres o Aprovisionadas con Programa Semanal
# OCO -> 4er Todas las Ordenes Liberadas o con Aprovisionamiento, con Numero de semana, de Campo
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'] == "X" ) | ((unir_reservaxorden_aviso['reservasolped'] == "3" ))) & (unir_reservaxorden_aviso['orden_revision'].str[0]== "C") & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Campo') , 250, unir_reservaxorden_aviso['ate_orden'])
# INO -> 1er Orden Los Inoperativos
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'] == "X" ) | ((unir_reservaxorden_aviso['reservasolped'] == "3" ))) & (unir_reservaxorden_aviso['orden_revision'].str[0]== "C") & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Campo') & (unir_reservaxorden_aviso['orden_texto'].str[0]== "I"), 210, unir_reservaxorden_aviso['ate_orden'])
# MAN -> 2er Orden Los Mantenimientos
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'] == "X" ) | ((unir_reservaxorden_aviso['reservasolped'] == "3" ))) & (unir_reservaxorden_aviso['orden_revision'].str[0]== "C") & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Campo') & (unir_reservaxorden_aviso['orden_texto'].str[0]== "M"), 220, unir_reservaxorden_aviso['ate_orden'])
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'] == "X" ) | ((unir_reservaxorden_aviso['reservasolped'] == "3" ))) & (unir_reservaxorden_aviso['orden_revision'].str[0]== "C") & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Mantenimiento Preventivo') & (unir_reservaxorden_aviso['orden_texto'].str[0] == 'M'), 220, unir_reservaxorden_aviso['ate_orden'])
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'] == "X" ) | ((unir_reservaxorden_aviso['reservasolped'] == "3" ))) & (unir_reservaxorden_aviso['orden_revision'].str[0]== "C") & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Mantenimiento SSGG') & (unir_reservaxorden_aviso['orden_texto'].str[0]== "M"), 220, unir_reservaxorden_aviso['ate_orden'])
# OCO -> 3er Orden Los Operativos con Observaciones
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'] == "X" ) | ((unir_reservaxorden_aviso['reservasolped'] == "3" ))) & (unir_reservaxorden_aviso['orden_revision'].str[0]== "C") & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Campo') & (unir_reservaxorden_aviso['orden_texto'].str[0]== "O"), 240, unir_reservaxorden_aviso['ate_orden'])

# Equipos -> Ordenes de trabajo Libres o Aprovisionadas sin Programa Semanal
# OCO -> 4er Todas las Ordenes Liberadas o con Aprovisionamiento, con Numero de semana, de Campo
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'] == "X" ) | ((unir_reservaxorden_aviso['reservasolped'] == "3" ))) & (~(unir_reservaxorden_aviso['orden_revision'].str[0]== "C")) & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Campo') , 300, unir_reservaxorden_aviso['ate_orden'])
# INO -> 1er Orden Los Inoperativos
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'] == "X" ) | ((unir_reservaxorden_aviso['reservasolped'] == "3" ))) & (~(unir_reservaxorden_aviso['orden_revision'].str[0]== "C")) & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Campo') & (unir_reservaxorden_aviso['orden_texto'].str[0]== "I"), 260, unir_reservaxorden_aviso['ate_orden'])
# MAN -> 2er Orden Los Mantenimientos
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'] == "X" ) | ((unir_reservaxorden_aviso['reservasolped'] == "3" ))) & (~(unir_reservaxorden_aviso['orden_revision'].str[0]== "C")) & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Campo') & (unir_reservaxorden_aviso['orden_texto'].str[0]== "M"), 270, unir_reservaxorden_aviso['ate_orden'])
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'] == "X" ) | ((unir_reservaxorden_aviso['reservasolped'] == "3" ))) & (~(unir_reservaxorden_aviso['orden_revision'].str[0]== "C")) & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Mantenimiento Preventivo') & (unir_reservaxorden_aviso['orden_texto'].str[0]== "M"), 270, unir_reservaxorden_aviso['ate_orden'])
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'] == "X" ) | ((unir_reservaxorden_aviso['reservasolped'] == "3" ))) & (~(unir_reservaxorden_aviso['orden_revision'].str[0]== "C")) & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Mantenimiento SSGG') & (unir_reservaxorden_aviso['orden_texto'].str[0]== "M"), 270, unir_reservaxorden_aviso['ate_orden'])
# OCO -> 3er Orden Los Operativos con Observaciones
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'] == "X" ) | ((unir_reservaxorden_aviso['reservasolped'] == "3" ))) & (~(unir_reservaxorden_aviso['orden_revision'].str[0]== "C")) & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Campo') & (unir_reservaxorden_aviso['orden_texto'].str[0]== "O"), 290, unir_reservaxorden_aviso['ate_orden'])

# Equipos -> Ordenes de trabajo NO Libres o NO Aprovisionadas con Programa Semanal
# OCO -> 4er Todas las Ordenes Liberadas o con Aprovisionamiento, con Numero de semana, de Campo
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'] == "" ) & ((unir_reservaxorden_aviso['reservasolped'] == "" ))) & (unir_reservaxorden_aviso['orden_revision'].str[0]== "C") & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Campo') , 950, unir_reservaxorden_aviso['ate_orden'])
# INO -> 1er Orden Los Inoperativos
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'] == "" ) & ((unir_reservaxorden_aviso['reservasolped'] == "" ))) & (unir_reservaxorden_aviso['orden_revision'].str[0]== "C") & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Campo') & (unir_reservaxorden_aviso['orden_texto'].str[0]== "I"), 910, unir_reservaxorden_aviso['ate_orden'])
# MAN -> 2er Orden Los Mantenimientos
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'] == "" ) & ((unir_reservaxorden_aviso['reservasolped'] == "" ))) & (unir_reservaxorden_aviso['orden_revision'].str[0]== "C") & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Campo') & (unir_reservaxorden_aviso['orden_texto'].str[0]== "M"), 920, unir_reservaxorden_aviso['ate_orden'])
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'] == "" ) & ((unir_reservaxorden_aviso['reservasolped'] == "" ))) & (unir_reservaxorden_aviso['orden_revision'].str[0]== "C") & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Mantenimiento Preventivo') & (unir_reservaxorden_aviso['orden_texto'].str[0]== "M"), 920, unir_reservaxorden_aviso['ate_orden'])
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'] == "" ) & ((unir_reservaxorden_aviso['reservasolped'] == "" ))) & (unir_reservaxorden_aviso['orden_revision'].str[0]== "C") & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Mantenimiento SSGG') & (unir_reservaxorden_aviso['orden_texto'].str[0]== "M"), 920, unir_reservaxorden_aviso['ate_orden'])
# OCO -> 3er Orden Los Operativos con Observaciones
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'] == "" ) & ((unir_reservaxorden_aviso['reservasolped'] == "" ))) & (unir_reservaxorden_aviso['orden_revision'].str[0]== "C") & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Campo') & (unir_reservaxorden_aviso['orden_texto'].str[0]== "O"), 940, unir_reservaxorden_aviso['ate_orden'])

# Equipos -> Ordenes de trabajo NO Libres o NO Aprovisionadas sin Programa Semanal
# OCO -> 4er Todas las Ordenes Liberadas o con Aprovisionamiento, con Numero de semana, de Campo
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'] == "" ) & ((unir_reservaxorden_aviso['reservasolped'] == "" ))) & (~(unir_reservaxorden_aviso['orden_revision'].str[0]== "C")) & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Campo') , 990, unir_reservaxorden_aviso['ate_orden'])
# INO -> 1er Orden Los Inoperativos
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'] == "" ) & ((unir_reservaxorden_aviso['reservasolped'] == "" ))) & (~(unir_reservaxorden_aviso['orden_revision'].str[0]== "C")) & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Campo') & (unir_reservaxorden_aviso['orden_texto'].str[0]== "I"), 960, unir_reservaxorden_aviso['ate_orden'])
# MAN -> 2er Orden Los Mantenimientos
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'] == "" ) & ((unir_reservaxorden_aviso['reservasolped'] == "" ))) & (~(unir_reservaxorden_aviso['orden_revision'].str[0]== "C")) & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Campo') & (unir_reservaxorden_aviso['orden_texto'].str[0]== "M"), 970, unir_reservaxorden_aviso['ate_orden'])
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'] == "" ) & ((unir_reservaxorden_aviso['reservasolped'] == "" ))) & (~(unir_reservaxorden_aviso['orden_revision'].str[0]== "C")) & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Mantenimiento Preventivo') & (unir_reservaxorden_aviso['orden_texto'].str[0]== "M"), 970, unir_reservaxorden_aviso['ate_orden'])
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'] == "" ) & ((unir_reservaxorden_aviso['reservasolped'] == "" ))) & (~(unir_reservaxorden_aviso['orden_revision'].str[0]== "C")) & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Mantenimiento SSGG') & (unir_reservaxorden_aviso['orden_texto'].str[0]== "M"), 970, unir_reservaxorden_aviso['ate_orden'])
# OCO -> 3er Orden Los Operativos con Observaciones
unir_reservaxorden_aviso['ate_orden'] = np.where( ((unir_reservaxorden_aviso['ot_liberado'] == "" ) & ((unir_reservaxorden_aviso['reservasolped'] == "" ))) & (~(unir_reservaxorden_aviso['orden_revision'].str[0]== "C")) & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Campo') & (unir_reservaxorden_aviso['orden_texto'].str[0]== "O"), 980, unir_reservaxorden_aviso['ate_orden'])

In [21]:
# Agrupar por Valor de Flota
lista_criterio1 = list(unir_reservaxorden_aviso["proyecto"])
lista_criterio2 = list(unir_reservaxorden_aviso["ate_orden"])

unir_reservaxorden_aviso['eqvaloradq_usd'] = unir_reservaxorden_aviso['eqvaloradq_usd'].replace('' , 0)

valor_flota = [unir_reservaxorden_aviso.loc[(unir_reservaxorden_aviso['proyecto'] == lista_criterio1[i]) & (unir_reservaxorden_aviso['ate_orden'] ==lista_criterio2[i]),"eqvaloradq_usd"].sum() for i in range(unir_reservaxorden_aviso.shape[0])]
# valor_flota.format("{:.2f}")
valor_flota = pd.DataFrame(valor_flota)
unir_reservaxorden_aviso["valor_flota"] = valor_flota

In [22]:
# STT
# ---

unir_reservaxorden_aviso['ate_orden'] = np.where( (unir_reservaxorden_aviso['ot_liberado'] == "")  & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Taller'), 999, unir_reservaxorden_aviso['ate_orden'])
# Equipos -> Ordenes de trabajo Libres con Programa Semanal
unir_reservaxorden_aviso['ate_orden'] = np.where( (unir_reservaxorden_aviso['ot_liberado'] == "X")  & (unir_reservaxorden_aviso['orden_revision'].str[0]== "C") & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Taller') & (unir_reservaxorden_aviso['orden_claact_deno'] == "Reparación"), 230, unir_reservaxorden_aviso['ate_orden'])
unir_reservaxorden_aviso['ate_orden'] = np.where( (unir_reservaxorden_aviso['ot_liberado'] == "X")  & (unir_reservaxorden_aviso['orden_revision'].str[0]== "C") & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Taller') & (unir_reservaxorden_aviso['orden_claact_deno'] == "Preparación"), 230, unir_reservaxorden_aviso['ate_orden'])
unir_reservaxorden_aviso['ate_orden'] = np.where( (unir_reservaxorden_aviso['ot_liberado'] == "X")  & (unir_reservaxorden_aviso['orden_revision'].str[0]== "C") & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Taller') & (unir_reservaxorden_aviso['orden_claact_deno'] == "Reparación venta"), 230, unir_reservaxorden_aviso['ate_orden'])
unir_reservaxorden_aviso['ate_orden'] = np.where( (unir_reservaxorden_aviso['ot_liberado'] == "X")  & (unir_reservaxorden_aviso['orden_revision'].str[0]== "C") & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Taller') & (unir_reservaxorden_aviso['orden_claact_deno'] == "Alistamiento"), 240, unir_reservaxorden_aviso['ate_orden'])

# Equipos -> Ordenes de trabajo Libres sin Programa Semanal
unir_reservaxorden_aviso['ate_orden'] = np.where( (unir_reservaxorden_aviso['ot_liberado'] == "X")  & (~(unir_reservaxorden_aviso['orden_revision'].str[0]== "C")) & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Taller') & (unir_reservaxorden_aviso['orden_claact_deno'] == "Reparación"), 280, unir_reservaxorden_aviso['ate_orden'])
unir_reservaxorden_aviso['ate_orden'] = np.where( (unir_reservaxorden_aviso['ot_liberado'] == "X")  & (~(unir_reservaxorden_aviso['orden_revision'].str[0]== "C")) & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Taller') & (unir_reservaxorden_aviso['orden_claact_deno'] == "Preparación"), 280, unir_reservaxorden_aviso['ate_orden'])
unir_reservaxorden_aviso['ate_orden'] = np.where( (unir_reservaxorden_aviso['ot_liberado'] == "X")  & (~(unir_reservaxorden_aviso['orden_revision'].str[0]== "C")) & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Taller') & (unir_reservaxorden_aviso['orden_claact_deno'] == "Reparación venta"), 280, unir_reservaxorden_aviso['ate_orden'])
unir_reservaxorden_aviso['ate_orden'] = np.where( (unir_reservaxorden_aviso['ot_liberado'] == "X")  & (~(unir_reservaxorden_aviso['orden_revision'].str[0]== "C")) & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Taller') & (unir_reservaxorden_aviso['orden_claact_deno'] == "Alistamiento"), 290, unir_reservaxorden_aviso['ate_orden'])

# Equipos -> Ordenes de trabajo NO Libres
unir_reservaxorden_aviso['ate_orden'] = np.where( (unir_reservaxorden_aviso['ot_liberado'] == "")  & (unir_reservaxorden_aviso['orden_revision'].str[0]== "C") & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Taller') & (unir_reservaxorden_aviso['orden_claact_deno'] == "Reparación"), 990, unir_reservaxorden_aviso['ate_orden'])
unir_reservaxorden_aviso['ate_orden'] = np.where( (unir_reservaxorden_aviso['ot_liberado'] == "")  & (unir_reservaxorden_aviso['orden_revision'].str[0]== "C") & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Taller') & (unir_reservaxorden_aviso['orden_claact_deno'] == "Preparación"), 990, unir_reservaxorden_aviso['ate_orden'])
unir_reservaxorden_aviso['ate_orden'] = np.where( (unir_reservaxorden_aviso['ot_liberado'] == "")  & (unir_reservaxorden_aviso['orden_revision'].str[0]== "C") & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Taller') & (unir_reservaxorden_aviso['orden_claact_deno'] == "Reparación venta"), 990, unir_reservaxorden_aviso['ate_orden'])
unir_reservaxorden_aviso['ate_orden'] = np.where( (unir_reservaxorden_aviso['ot_liberado'] == "")  & (unir_reservaxorden_aviso['orden_revision'].str[0]== "C") & (unir_reservaxorden_aviso['oden_clase_deno'] == 'Reparación Taller') & (unir_reservaxorden_aviso['orden_claact_deno'] == "Alistamiento"), 990, unir_reservaxorden_aviso['ate_orden'])

# 2.3 Equipamiento y Otros

# Reservas para Centro de Costos
unir_reservaxorden_aviso['ate_orden'] = np.where( (unir_reservaxorden_aviso['orden'] == ''), 500, unir_reservaxorden_aviso['ate_orden'])
# Ordenes de Proyecto: 1 (113xxxx -> Proyectos de Mejora / 114xxxxx -> Compra de Activos / 18xxx -> Proyectos Campo) 
unir_reservaxorden_aviso.loc[unir_reservaxorden_aviso['orden'].str[0] == "1" , 'ate_orden'] = 600
# Ordenes de Proyecto: 2
unir_reservaxorden_aviso.loc[unir_reservaxorden_aviso['orden'].str[0] == "2" , 'ate_orden'] = 620
# Ordenes de Capacitacion: 7
unir_reservaxorden_aviso.loc[unir_reservaxorden_aviso['orden'].str[0] == "7" , 'ate_orden'] = 640

# Las reservas que no se vana utilizar (ate_accion = "FIN"), colocar 999
unir_reservaxorden_aviso.loc[unir_reservaxorden_aviso['ate_accion'] == "FIN" , 'ate_orden'] = 999
unir_reservaxorden_aviso.loc[unir_reservaxorden_aviso['ate_orden'] == 999 , 'ate_accion'] = "FIN"

In [23]:
unir_reservaxorden_aviso['material'] = unir_reservaxorden_aviso['material'].str.replace(' ','')
tabla_material['material'] = tabla_material['material'].str.replace(' ','')

In [24]:
reservaxorden_material = unir_reservaxorden_aviso.merge(tabla_material,how='left',on='material')

In [25]:
reservaxorden_material['identificador']=(reservaxorden_material['orden']+reservaxorden_material['reserva']+reservaxorden_material['reservapos'])

In [26]:
reservaxorden_material = reservaxorden_material.sort_values(by = ['ate_orden', 'valor_flota','fe_nece', 'reserva', 'reservapos'], ascending = [True, False, True, True, True], na_position = 'last').reset_index(drop=True)
reservaxorden_material['index'] = reservaxorden_material.index

In [27]:
display(reservaxorden_material)

,reserva,reservapos,material,materialdeno,ctd_nec,ctd_reduc,ctd_dif,um_base,orden,usuario,...,Ped_reserva,Ped_nro_necesidad,Ped_proveedor,estadoreserva,Cod_mate_antiguo,Grp_mat,Tp_mat,Jerarquia_produc,identificador,index
0,11471,1,EPP0000011,GUANTES DE BADANA,2.00,0.00,2.0,UND,P43178070,NBUSTAMANTE,...,,,,,EPP0000011,G041,ZMER,0103E020241,P43178070114711,0
1,11471,10,EPP0000009,CASCO DE SEGURIDAD,1.00,0.00,1.0,UND,P43178070,NBUSTAMANTE,...,,,,,EPP0000009,G041,ZMER,0103E020239,P431780701147110,1
2,11471,11,UNI0000002,CHALECO TALLA M,1.00,0.00,1.0,UND,P43178070,NBUSTAMANTE,...,,,,,UNI0000002,G057,ZMER,0310I020370,P431780701147111,2
3,11471,12,UNI0000027,UNIFORME ESTANDAR TALLA M - VERDE,2.00,0.00,2.0,UND,P43178070,NBUSTAMANTE,...,,,,,UNI0000027,G057,ZMER,0310I020370,P431780701147112,3
4,11471,13,UNI0000024,POLO DE MANGA LARGA TALLA M VERDE,2.00,0.00,2.0,UND,P43178070,NBUSTAMANTE,...,,,,,UNI0000024,G057,ZMER,0310I020370,P431780701147113,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9744,38392,1,HAWX3264988,KIT DE TORNILLOS,1.00,0.00,1.0,UND,4000016894,GGALLEGOS,...,,,,Lib Mov.Permitdo,WX3264988,G020,ZMER,0101C020055,4000016894383921,9744
9745,38392,2,HAWX3263186,EXCENTRICA SISTEMA DE VIBRAC,1.00,0.00,1.0,UND,4000016894,GGALLEGOS,...,,,,Lib Mov.Permitdo,WX3263186,G128,ZMER,0101T030229,4000016894383922,9745
9746,38405,1,1300001043,Mntto y Rep. Vehiculos Leasing,1.00,0.00,1.0,ZZ,4000016901,DRUFINO,...,,,,Lib Mov.Permitdo,,S018,ZSER,0414S270438,4000016901384051,9746
9747,37917,1,HIXB00017316,COOLER,1.00,0.00,1.0,UND,4000016737,FMOTTA,...,,,,,XB00017316,G082,ZMER,0101M020173,4000016737379171,9747


In [28]:
selec_reservaxmaterial = reservaxorden_material[['index','identificador','material','ate_accion','ctd_dif','stock_asignado','trasl_ctd_c154','trasl_ctd_c040',
                                                   'trasl_ctd_c080','trasl_ctd_c200','trasl_ctd_c152','trasl_fecha','Ped_nro','Ped_Fecha','Ped_ctd',
                                                   'Ped_reserva','Ped_nro_necesidad','Ped_proveedor']]

In [29]:
filtrar_reservas = selec_reservaxmaterial.loc[(selec_reservaxmaterial['ate_accion']=='MERCADERIA') & (selec_reservaxmaterial['ctd_dif']>0)]